In [25]:
import config
import requests
import json
import numpy as np
import pandas as pd

In [26]:
# Pass in as arg {0} for the URL, referenced as method_dict[key]
method_dict = {"GetNewsForApp":"ISteamNews/GetNewsForApp/v0002/", 
             "GetGlobalAchievementPercentagesForApp": "ISteamUserStats/GetGlobalAchievementPercentagesForApp/v0002/", 
             "GetGlobalStatsForGame": "ISteamUserStats/GetGlobalStatsForGame/v0001/", 
             "GetPlayerSummaries": "ISteamUser/GetPlayerSummaries/v0002/", 
             "GetFriendList": "ISteamUser/GetFriendList/v0001/", 
             "GetPlayerAchievements": "ISteamUserStats/GetPlayerAchievements/v0001/", 
             "GetUserStatsForGame": "ISteamUserStats/GetUserStatsForGame/v0002/", 
             "GetOwnedGames": "IPlayerService/GetOwnedGames/v0001/", 
             "GetRecentlyPlayedGames": "IPlayerService/GetRecentlyPlayedGames/v0001/", 
             "IsPlayingSharedGame": "IPlayerService/IsPlayingSharedGame/v0001/", 
             "GetSchemaForGame": "ISteamUserStats/GetSchemaForGame/v2/", 
             "GetPlayerBans": "ISteamUser/GetPlayerBans/v1/"
            }

api_method = method_dict["GetOwnedGames"]

# The required api_args are different depending on which method is being called.
# For simplicity's sake, the api_args currently is just steam_id.
# This limits the number of methods that are currently usable.
# Expanding the number of usable methods is not a pressing issue, but will be addressed after minimum viable functionality is achieved.

# This is my own Steam ID. This script must not be used to query another user's data without permission.
steam_id = "&steamid=76561197972309962"
api_args = steam_id

fetch_url = "http://api.steampowered.com/{0}?key={1}{2}".format(api_method, config.api_key, api_args)

r = requests.get(fetch_url)
r.json()

{'response': {'game_count': 224,
  'games': [{'appid': 10, 'playtime_forever': 0},
   {'appid': 20, 'playtime_forever': 0},
   {'appid': 30, 'playtime_forever': 0},
   {'appid': 40, 'playtime_forever': 0},
   {'appid': 50, 'playtime_forever': 0},
   {'appid': 60, 'playtime_forever': 0},
   {'appid': 70, 'playtime_forever': 1326},
   {'appid': 130, 'playtime_forever': 0},
   {'appid': 220, 'playtime_forever': 1434},
   {'appid': 240, 'playtime_forever': 1407},
   {'appid': 320, 'playtime_forever': 0},
   {'appid': 340, 'playtime_forever': 0},
   {'appid': 300, 'playtime_forever': 0},
   {'appid': 1300, 'playtime_forever': 0},
   {'appid': 1309, 'playtime_forever': 0},
   {'appid': 1313, 'playtime_forever': 0},
   {'appid': 2800, 'playtime_forever': 20},
   {'appid': 6910, 'playtime_forever': 65},
   {'appid': 2590, 'playtime_forever': 0},
   {'appid': 400, 'playtime_forever': 562},
   {'appid': 12140, 'playtime_forever': 128},
   {'appid': 12150, 'playtime_forever': 0},
   {'appid': 121